Run this code to view the upload button. Upload your excel file by clicking the upload button. The Upload(0) should change to Upload(1). The script will only work with the correct file so if you recieve error messages please let us know.

In [8]:
import io
import pandas as pd
from ipywidgets import FileUpload
import networkx as nx
import math
upload = FileUpload(accept='.xlsx', multiple=False)

In [10]:
upload

FileUpload(value=({'name': 'projectNetwork.xlsx', 'type': 'application/vnd.openxmlformats-officedocument.sprea…

Once uploaded, run the following script to generate a GEXF file for you network

In [13]:
def makeGraph(df, actors, graphName):

    try:
        df = df.drop(['total'])
        df = df.drop(columns=['total'])
    except:
        pass
    
    nodeDict = {}
    for k,v in actors.iterrows():
        # print(k)
        nodeDict['{k}_{g}'.format(k=k, g=graphName)] = {
            'name': k,
            'role': v['Role'],
            'weight': [],
        }
    
    edgeDict = {}
    cols = list(df)
    names = []
    for col in cols:
        try:
            col = float(col)
        except ValueError:
            if col != 'total':
                names.append(col)
   
    for k, v in df.iterrows():
        if k in names:
            n1 = '{k}_{g}'.format(k=k, g=graphName)
            
            for name in names:
                n2 = '{k}_{g}'.format(k=name, g=graphName)
                
                if n1 != n2:
                    edge = tuple(sorted([n1, n2]))
                    edgeWeight = v[name]
                          
                    if edge not in edgeDict.keys():
                        edgeDict[edge] = {
                            'weight': []
                        }

                    if math.isnan(edgeWeight) == True:
                        edgeWeight = 0
                    else:
                        edgeDict[edge]['weight'].append(edgeWeight)   
                        nodeDict[n1]['weight'].append(edgeWeight)
                        nodeDict[n2]['weight'].append(edgeWeight)
 
    return nodeDict, edgeDict

data = upload.value[0]
dfActor = pd.read_excel(io.BytesIO(data['content']), sheet_name='Actors', header=0, index_col=0)

# make graph
G = nx.Graph()

for name in ('P1', 'P2', 'P3'):
    df = pd.read_excel(io.BytesIO(data['content']), sheet_name='Point{n}'.format(n=name[-1]), header=0, index_col=0)
    nodeDict, edgeDict = makeGraph(df, dfActor, name)
    
    for n in nodeDict.keys():
        weight = sum(nodeDict[n]['weight']) / len(nodeDict[n]['weight'])
        name = nodeDict[n]['name']
        role = nodeDict[n]['role']
        G.add_node(n, weight=weight, name=name, role=role)

    for e in edgeDict.keys():
        weight = sum(edgeDict[e]['weight']) / len(edgeDict[e]['weight'])
        if weight > 0:
            G.add_edge(e[0], e[1], weight=weight)

# write file
filename = data['name']
filename = '{f}_GEXF.gexf'.format(f=filename.split('.')[0])
nx.write_gexf(G, filename)

print('done!')

done!
